In [16]:
import numpy as np
import pandas as pd
import glob
from model.ctabgan import CTAB_XTRA_DP
df = pd.read_csv("datasets/car.csv")
df = df.drop(columns=['Year','Model'])

In [17]:

#df.iloc[0, 1] = np.nan
df = df[['Brand', 'Engine_Size', 'Mileage']]
df.head(10)

,Brand,Engine_Size,Mileage
0,Kia,4.2,289944
1,Chevrolet,2.0,5356
2,Mercedes,4.2,231440
3,Audi,2.0,160971
4,Volkswagen,2.6,286618
5,Toyota,2.7,157889
6,Honda,3.4,139584
7,Kia,4.7,157495
8,Kia,2.6,98700
9,Toyota,3.1,107724


In [20]:
synthesizer =  CTAB_XTRA_DP(df,
                 categorical_columns = ["Brand"], 
                 log_columns = {}, # Fuck this log, task for tommorrow
                 mixed_columns= {"Engine_Size": [np.nan]},
                 gaussian_columns = [""],
                 integer_columns = []) 

In [21]:
synthesizer.fit(10)

Start training with 10 epochs
 5 critic iterations
 20 steps per epoch
 batch size 500


100%|██████████| 10/10 [01:54<00:00, 11.49s/it]


In [24]:
syn = synthesizer.generate_samples(1000)
syn.head(20)


,Brand,Engine_Size,Mileage
0,BMW,1.1,58121
1,Hyundai,3.9,138564
2,Chevrolet,NaN,173569
3,Chevrolet,4.7,278335
4,Ford,3.5,194690
5,Mercedes,3.5,112590
6,Honda,2.4,314200
7,Honda,4.4,238866
8,Honda,4.3,138564
9,Hyundai,4.6,278337


In [25]:
from model.eval.evaluation import privacy_metrics, stat_sim

privacy = privacy_metrics(df,syn)

==== Privacy Metrics Analysis ====
Using 15% of data for analysis
Real data sample size: 1500
Fake data sample size: 150

Calculating pairwise distances...

== Nearest Neighbor Ratio Metrics (5th percentile) ==
Real-to-Real NN Ratio (5th): 0.1161
Fake-to-Fake NN Ratio (5th): 0.0044
Real-to-Fake NN Ratio (5th): 0.2865

== Minimum Distance Metrics (5th percentile) ==
Real-to-Fake Min Distance (5th): 0.0137
Real-to-Real Min Distance (5th): 0.0022
Fake-to-Fake Min Distance (5th): 0.0003

Privacy Risk Score: 11.1993
(Higher score indicates better privacy protection)

==== Summary Interpretation ====
✓ Good distance between real and synthetic records
✓ Good neighbor distance ratios


In [27]:
sim = stat_sim(df,syn,"Brand")

column:  Brand JSD:  0.05781818373035923
column:  Engine_Size WD:  nan
column:  Mileage WD:  0.03548415388000882
